In [1]:
!pip install ipywidgets
!pip install datasets
!pip install huggingface_hub
!pip install pyarrow==15.0.2

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time
import json
import os
from datasets import load_dataset

In [4]:
url = 'https://www.house.kg/kupit-kvartiru'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
pages = soup.find_all("li", class_="page-item")[-1].find("a", class_="page-link").get("data-page")
pages

'651'

In [9]:
with open(f'house_kg.json', 'a', encoding="utf-8") as f:
  for page in tqdm(range(1, int(pages)+1), leave=True):
    url = f'https://www.house.kg/kupit-kvartiru?page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    objects = soup.find("div", class_="listings-wrapper")
    lincs = objects.find_all('div', class_="left-image")
    for ur in tqdm(lincs, leave=False):
      det_url = "https://www.house.kg" + ur.find('a')['href']
      response = requests.get(det_url)
      soup = BeautifulSoup(response.text, "html.parser")
      atr_dict = {}
      atr_dict['url'] = det_url

      atr_dict['usd_price'] = soup.find("div", class_="price-dollar").text.strip().replace(' ', '').replace('$', '')
      atr_dict['kgs_price'] = soup.find("div", class_="price-som").text.strip().replace(' ', '').replace('сом', '')
      atr_dict['Комнаты'] = soup.find('div', class_='left').find('h1').text.strip()[0]
      atr_dict['Адрес'] = soup.find('div', class_='left').find('div', class_='address').text.strip()
      atr_dict['Телефон_продавца'] = soup.find("div", class_="number").text
      atr_dict['Добавлено'] = soup.find('span', class_ = 'added-span').text
      atr_dict['Просмотры'] = soup.find('span', class_ = 'view-count').text.strip()
      try:
        atr_dict['Поднято'] = soup.find('span', class_ = 'upped-span').text.strip()
      except Exception:
        pass
      try:
        atr_dict['Описание'] = soup.find("div", class_="description").find('p').text
      except Exception:
        pass
      try:
        atr_dict['Рейтинг'] = soup.find('div', class_ = 'rating score').text.strip()
      except Exception:
        pass
      try:
        atr_dict['Лайки'] = soup.find('span', class_ = 'favourite-count table-comments').text.strip()
      except Exception:
        pass
      atr_dict['Долгота'] = soup.find("div", class_="map-wrapper").find("div", id="map2gis").get("data-lon")
      atr_dict['Широта'] = soup.find("div", class_="map-wrapper").find("div", id="map2gis").get("data-lat")
      atributs = soup.find_all("div", class_="info-row")
      for atr in atributs:
        key = atr.find('div', class_="label").text.strip().replace('\n', ' ')
        val = atr.find('div', class_="info").text.strip().replace('\n', ' ')
        atr_dict[key] = val

      json.dump(atr_dict, f, ensure_ascii=False)
      f.write('\n')

  0%|          | 0/651 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
for page in tqdm(range(1, int(pages)+1), leave=True):
    url = f'https://www.house.kg/kupit-kvartiru?page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    objects = soup.find("div", class_="listings-wrapper")
    lincs = objects.find_all('div', class_="left-image")
    for ur in tqdm(lincs, leave=False):
      det_url = "https://www.house.kg" + ur.find('a')['href']
      response = requests.get(det_url)
      soup = BeautifulSoup(response.text, "html.parser")
      images = soup.find("div", class_="fotorama").find_all('a')[:-1]
      for img in tqdm(images, leave=False):
        image = requests.get(img['href']).content
        filename = f'images/{det_url.split("/")[-1]}/{img["href"].split("/")[-1]}'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'wb') as f:
          f.write(image)

  0%|          | 0/651 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
df = pd.read_json('house_kg.json', lines=True)

In [11]:
df.to_csv('house_kg.csv', index=False)

In [12]:
df.shape

(3010, 41)

In [13]:
df.head()

,url,usd_price,kgs_price,Комнаты,Адрес,Телефон_продавца,Добавлено,Просмотры,Поднято,Описание,...,Мебель,Пол,Безопасность,Возможность рассрочки,Возможность ипотеки,Возможность обмена,Площадь участка,Канализация,Питьевая вода,Электричество
0,https://www.house.kg/details/932931666cc8a829e...,97000,8184860,3,"Бишкек, Магистраль, Магистраль/Сухэ Батора",+996 500 07-57-77,Добавлено 25 дней назад,4376,Поднято 1 час назад,В срочной продаже 3-х комнатная квартира студи...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.house.kg/details/521743066ec024c57...,220000,18563600,4,"Бишкек, Парк Панфилова/Спартак, Панфилова 190/1",+996 557 50-55-75,Добавлено 1 день назад,1118,Поднято 7 часов назад,Продается 4 комнатная квартира по срочной цене...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.house.kg/details/141579566ed52b0b8...,85000,7172300,1,"Бишкек, Магистраль, Сухэ-Батора 24/Токомбаева",+996 706 89-49-26,Добавлено 34 минуты назад,41,Поднято 26 минут назад,Продаётся 2 комнатная студийная квартира в рай...,...,полностью меблирована,паркет,"сигнализация, ...",нет,нет,NaN,NaN,NaN,NaN,NaN
3,https://www.house.kg/details/757010366ed4f9839...,85500,7214490,2,"Бишкек, Магистраль, Сухэ-Батора 24/Токомбаева",+996 995 56-55-56,Добавлено 48 минут назад,33,Поднято 26 минут назад,Продаётся 2 комнатная студийная квартира в рай...,...,полностью меблирована,паркет,"охрана, ...",NaN,NaN,обмен не предлагать,NaN,NaN,NaN,NaN
4,https://www.house.kg/details/957794666ebe80157...,100000,8438000,4,"Бишкек, Средний Джал м-н, Джал 23 16/Тыналиева",+996 500 55-54-54,Добавлено 1 день назад,579,Поднято 1 час назад,‼️Срочно ‼️\r\n🏢 Продаю 4 - комнатую квартиру ...,...,частично меблирована,NaN,"решетки на окнах, ...",NaN,есть,NaN,NaN,NaN,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3010 entries, 0 to 3009
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   url                             3010 non-null   object 
 1   usd_price                       3010 non-null   int64  
 2   kgs_price                       3010 non-null   int64  
 3   Комнаты                         3010 non-null   int64  
 4   Адрес                           3010 non-null   object 
 5   Телефон_продавца                3010 non-null   object 
 6   Добавлено                       3010 non-null   object 
 7   Просмотры                       3010 non-null   int64  
 8   Поднято                         2903 non-null   object 
 9   Описание                        2936 non-null   object 
 10  Лайки                           2013 non-null   float64
 11  Долгота                         3010 non-null   float64
 12  Широта                          30

In [18]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [26]:
dataset = load_dataset("imagefolder", data_dir="./images")

Resolving data files:   0%|          | 0/242 [00:00<?, ?it/s]

In [27]:
dataset.push_to_hub("sharshenaliev/images")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sharshenaliev/images/commit/75b0e4c75b6cd057cc65b9adca4365fb40dc6304', commit_message='Upload dataset', commit_description='', oid='75b0e4c75b6cd057cc65b9adca4365fb40dc6304', pr_url=None, pr_revision=None, pr_num=None)

In [24]:
dataset = load_dataset("csv", data_files="house_kg.csv")

In [25]:
dataset.push_to_hub("sharshenaliev/house_kg")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sharshenaliev/house_kg/commit/c5f33b5e582fa9f06509ae366b9ea6d9c5dd5466', commit_message='Upload dataset', commit_description='', oid='c5f33b5e582fa9f06509ae366b9ea6d9c5dd5466', pr_url=None, pr_revision=None, pr_num=None)